# Load Streamflow ML results

## filter q (bfd=1)

In [1]:
import pandas as pd
import os

# from main_jupyter import final_measurements_delta

# Define the directory containing the CSV files
directory = '../data/raw/streamflow/GSLB_ML'

# Initialize an empty DataFrame to store the results
compiled_data = pd.DataFrame(columns=['gage_id', 'date', 'q', 'bfd'])

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(directory, filename)

        # Read the CSV file
        df = pd.read_csv(file_path)

        # Filter rows where ML_BFD is 1
        filtered_df = df[df['ML_BFD'] == 1]

        # Extract gage_id from the filename (assuming filename is the gage_id)
        gage_id = os.path.splitext(filename)[0]

        # Add a new column for gage_id
        filtered_df['gage_id'] = gage_id

        # Select and rename the necessary columns
        filtered_df = filtered_df[['gage_id', 'date','Q', 'ML_BFD']]
        filtered_df.columns = ['gage_id', 'date', 'q', 'bfd']

        # Append to the compiled DataFrame
        compiled_data = pd.concat([compiled_data, filtered_df], ignore_index=True)

# Display the compiled DataFrame
compiled_data


/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_12160/2827158917.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['gage_id'] = gage_id
/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_12160/2827158917.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  compiled_data = pd.concat([compiled_data, filtered_df], ignore_index=True)
/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_12160/2827158917.py:28: SettingWithCopyWarning: 
A value is trying to be set on a

,gage_id,date,q,bfd
0,10015900,1958-04-01,0.0,1.0
1,10015900,1958-04-02,0.0,1.0
2,10015900,1958-04-03,0.0,1.0
3,10015900,1958-04-04,0.0,1.0
4,10015900,1958-04-05,0.0,1.0
...,...,...,...,...
900051,10058600,1986-09-10,35.0,1.0
900052,10058600,1986-09-14,33.6,1.0
900053,10058600,1986-09-24,32.9,1.0
900054,10058600,1986-09-25,34.4,1.0


## streamflow outliers

In [8]:
import pandas as pd
import numpy as np
from scipy import stats


class SimpleOutlierDetector:
    """Simplified outlier detection class"""

    def __init__(self, data, column):
        self.data = data.copy()
        self.column = column
        self.results = None

    def detect_outliers(self, zscore_threshold=3.0, iqr_multiplier=1.5):
        """Detect outliers"""
        data = self.data.copy()
        # Initialize outlier flag columns
        data['is_outlier_zscore'] = False
        data['is_outlier_iqr'] = False

        # 1. Z-score method
        try:
            z_scores = np.abs(stats.zscore(data[self.column], nan_policy='omit'))
            data['is_outlier_zscore'] = z_scores > zscore_threshold
        except:
            pass

        # 2. IQR method
        try:
            Q1 = np.nanpercentile(data[self.column], 25)
            Q3 = np.nanpercentile(data[self.column], 75)
            IQR = Q3 - Q1
            lower_bound = Q1 - iqr_multiplier * IQR
            upper_bound = Q3 + iqr_multiplier * IQR
            data['is_outlier_iqr'] = (data[self.column] < lower_bound) | (data[self.column] > upper_bound)
        except:
            pass

        # Combined outlier detection
        data['is_outlier_any'] = data[['is_outlier_zscore', 'is_outlier_iqr']].any(axis=1)

        self.results = data
        return self.results

    def get_clean_data(self):
        """Get clean data"""
        if self.results is None:
            raise ValueError("Please run detect_outliers() method first")
        return self.results[~self.results['is_outlier_any']].copy()


# Usage example
detector = SimpleOutlierDetector(compiled_data, 'q')
outlier_results = detector.detect_outliers()
clean_data = detector.get_clean_data()

# Display clean data
print(clean_data.head())




    gage_id        date    q  bfd  is_outlier_zscore  is_outlier_iqr  \
0  10015900  1958-04-01  0.0  1.0              False           False   
1  10015900  1958-04-02  0.0  1.0              False           False   
2  10015900  1958-04-03  0.0  1.0              False           False   
3  10015900  1958-04-04  0.0  1.0              False           False   
4  10015900  1958-04-05  0.0  1.0              False           False   

   is_outlier_any  
0           False  
1           False  
2           False  
3           False  
4           False  


In [9]:
# Get detection results
outlier_results = detector.detect_outliers()

# Display total number of outliers detected
total_outliers = outlier_results['is_outlier_any'].sum()
print(f"Total number of outliers detected: {total_outliers}")

# Display number of outliers detected by each method
zscore_outliers = outlier_results['is_outlier_zscore'].sum()
iqr_outliers = outlier_results['is_outlier_iqr'].sum()

print(f"Number of outliers detected by Z-score method: {zscore_outliers}")
print(f"Number of outliers detected by IQR method: {iqr_outliers}")

# Mark records detected as outliers by both methods
outlier_results['is_outlier_both'] = outlier_results['is_outlier_zscore'] & outlier_results['is_outlier_iqr']

# Remove records that are outliers by both methods
clean_data = outlier_results[~outlier_results['is_outlier_both']].copy()

# Display summary of data cleaning
removed_count = outlier_results['is_outlier_both'].sum()
print(f"Number of outliers removed (detected by both methods): {removed_count}")
print(f"Number of records after cleaning: {len(clean_data)}")


Total number of outliers detected: 35377
Number of outliers detected by Z-score method: 14340
Number of outliers detected by IQR method: 35377
Number of outliers removed (detected by both methods): 14340
Number of records after cleaning: 885716


In [10]:
clean_data=clean_data[['gage_id','date','q','bfd']]

In [12]:
compiled_data=clean_data.copy()
compiled_data.head()

,gage_id,date,q,bfd
0,10015900,1958-04-01,0.0,1.0
1,10015900,1958-04-02,0.0,1.0
2,10015900,1958-04-03,0.0,1.0
3,10015900,1958-04-04,0.0,1.0
4,10015900,1958-04-05,0.0,1.0


In [13]:
# Define the output directory
output_directory = '../data/processed/streamflow'
os.makedirs(output_directory, exist_ok=True)

# Define the output file path
output_file_path = os.path.join(output_directory, 'q_bfd_1.csv')

# Save the compiled DataFrame to a CSV file
compiled_data.to_csv(output_file_path, index=False)

# Display the path where the file is saved
output_file_path


'../data/processed/streamflow/q_bfd_1.csv'

# PCHIP well wte

In [14]:
import pandas as pd
well_ts = pd.read_csv('../data/raw/groundwater/GSLB_1900-2023_TS_with_aquifers.csv')
well_ts.columns = well_ts.columns.str.lower()
well_ts.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177209 entries, 0 to 177208
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   aquiferid  177209 non-null  int64  
 1   well_id    177209 non-null  int64  
 2   date       177209 non-null  object 
 3   wte        177209 non-null  float64
 4   state      177209 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 6.8+ MB


In [15]:
import pandas as pd


def analyze_well_time_spans(well_ts):
    """
    Analyze well time spans and calculate key statistics
    Args:
        well_ts: Dictionary with well data containing start_date, end_date and data_points
    Returns:
        DataFrame with time span statistics
    """
    data = []
    for well_id, well_data in well_ts.items():
        start_date = pd.to_datetime(well_data['start_date'])
        end_date = pd.to_datetime(well_data['end_date'])
        time_span = (end_date - start_date).days / 365.0
        density = well_data['data_points'] / time_span if time_span > 0 else 0
        data.append({
            "well_id": well_id,
            "time_span_years": time_span,
            "data_density": density,
        })
    return pd.DataFrame(data)


def generate_time_span_report(df):
    """
    Generate summary report from well statistics DataFrame
    Args:
        df: DataFrame with well statistics
    """
    print(f"\nSummary Statistics:")
    print(f"- Total wells: {len(df):,}")
    if 'time_span_years' in df:
        print(f"- Time span range: {df['time_span_years'].min():.1f} - {df['time_span_years'].max():.1f} years")
        print(f"- Average time span: {df['time_span_years'].mean():.1f} years")
    if 'data_density' in df:
        print(f"- Data density range: {df['data_density'].min():.1f} - {df['data_density'].max():.1f} points/year")


# Example usage
well_ts = {
    "well_1": {"start_date": "2015-01-01", "end_date": "2020-01-01", "data_points": 50},
    "well_2": {"start_date": "2010-06-15", "end_date": "2022-06-15", "data_points": 100}
}

stats_df = analyze_well_time_spans(well_ts)
generate_time_span_report(stats_df)



Summary Statistics:
- Total wells: 2
- Time span range: 5.0 - 12.0 years
- Average time span: 8.5 years
- Data density range: 8.3 - 10.0 points/year


## detect outliers

In [40]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings

warnings.filterwarnings('ignore')


class SimpleGroundwaterOutlierDetector:
    """Simplified groundwater data outlier detection class - for interpolation preparation"""
    DATE_COLUMN = 'date'
    WELL_ID_COLUMN = 'well_id'

    def __init__(self, data):
        self.data = data.copy()
        self._validate_columns()
        # Convert date column during initialization
        self.data[self.DATE_COLUMN] = pd.to_datetime(self.data[self.DATE_COLUMN], errors='coerce')
        self.results = None

    def _validate_columns(self):
        """Validate that required columns exist in the data"""
        required_columns = [self.DATE_COLUMN, self.WELL_ID_COLUMN]
        missing_columns = [col for col in required_columns if col not in self.data.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")

    def detect_outliers(self, min_points=5, zscore_threshold=3.0, iqr_multiplier=2):
        """
        Detect outliers using statistical methods
        
        Args:
            min_points: Minimum points needed for statistical tests 
            zscore_threshold: Z-score threshold for outlier detection
            iqr_multiplier: IQR multiplier for outlier detection
        """
        # Sort by well_id and date
        self.data = self.data.sort_values([self.WELL_ID_COLUMN, self.DATE_COLUMN]).reset_index(drop=True)
        results = []

        for well_id in self.data['well_id'].unique():
            well_data = self.data[self.data['well_id'] == well_id].copy()
            n_points = len(well_data)

            well_data['is_outlier'] = False

            if n_points >= min_points:
                wte_values = well_data['wte'].values

                # Z-score method
                z_scores = np.abs(stats.zscore(wte_values, nan_policy='omit'))
                is_zscore_outlier = z_scores > zscore_threshold

                # IQR method
                Q1, Q3 = np.nanpercentile(wte_values, [25, 75])
                IQR = Q3 - Q1
                if IQR > 0:
                    lower_bound = Q1 - iqr_multiplier * IQR
                    upper_bound = Q3 + iqr_multiplier * IQR
                    is_iqr_outlier = (wte_values < lower_bound) | (wte_values > upper_bound)

                    # Combine methods
                    well_data['is_outlier'] = is_zscore_outlier | is_iqr_outlier
                results.append(well_data)

        if results:
            self.results = pd.concat(results, ignore_index=True)
        return self.results

    def get_clean_data(self):
        """Get clean data suitable for interpolation"""
        if self.results is None:
            return None

        clean_data = self.results[~self.results['is_outlier']].copy()

        # Print interpolation readiness stats
        well_stats = clean_data.groupby('well_id').size()
        print(f"\nInterpolation readiness summary:")
        print(f"- Wells with no data: {(well_stats == 0).sum()}")
        print(f"- Wells with 1-2 points: {((well_stats >= 1) & (well_stats <= 2)).sum()}")
        print(f"- Wells with 3+ points: {(well_stats >= 3).sum()} (suitable for PCHIP)")

        return clean_data


def clean_well_data_for_interpolation(well_ts, min_points=5):
    """Main function to clean groundwater data for interpolation"""
    detector = SimpleGroundwaterOutlierDetector(well_ts)
    detector.detect_outliers(min_points=min_points)
    return detector.get_clean_data()


def get_strictly_clean_data(detector):
    """
    Extract strictly clean data by removing values detected as outliers by all methods
    Returns clean_df and removed_df
    """
    if detector.results is None:
        raise ValueError("Please run detect_outliers() first")

    df = detector.results.copy()

    # Ensure required columns exist
    required_cols = ['is_outlier_zscore', 'is_outlier_iqr', 'is_outlier_modified_zscore']
    if not all(col in df.columns for col in required_cols):
        raise ValueError("Missing required outlier detection columns")

    # Strong outliers detected by all methods
    df['is_strong_outlier'] = df['is_outlier_zscore'] & df['is_outlier_iqr'] & df['is_outlier_modified_zscore']

    # Split into clean and removed data
    clean_df = df[~df['is_strong_outlier']].copy()
    removed_df = df[df['is_strong_outlier']].copy()

    print(f"Strong outliers removed:")
    print(f"- Total records removed: {len(removed_df):,}")
    print(f"- Clean records remaining: {len(clean_df):,}")


    return clean_df, removed_df

In [41]:
# Reload the well_ts DataFrame since it was overwritten by the dictionary example
well_ts = pd.read_csv('../data/raw/groundwater/GSLB_1900-2023_TS_with_aquifers.csv')
well_ts.columns = well_ts.columns.str.lower()

# Now use the actual well_ts DataFrame for outlier detection
clean_data = clean_well_data_for_interpolation(well_ts)


Interpolation readiness summary:
- Wells with no data: 0
- Wells with 1-2 points: 0
- Wells with 3+ points: 2338 (suitable for PCHIP)


## PCHIP wte

In [43]:
import pandas as pd
from scipy.interpolate import PchipInterpolator

# Convert all column names to lowercase
clean_data.columns = clean_data.columns.str.lower()

# Convert date column to datetime format
clean_data['date'] = pd.to_datetime(clean_data['date'])

# Count data points for each well
well_counts = clean_data['well_id'].value_counts()
total_wells = len(well_counts)
wells_with_one_point = (well_counts == 1).sum()
wells_with_two_points = (well_counts == 2).sum()

# Print summary statistics
print(f"Total number of wells: {total_wells}")
print(f"Number of wells with only one data point: {wells_with_one_point}")
print(f"Number of wells with only two data points: {wells_with_two_points}")


Total number of wells: 2338
Number of wells with only one data point: 0
Number of wells with only two data points: 0


In [44]:
import pandas as pd
import numpy as np
from scipy.interpolate import PchipInterpolator


def interpolate_daily_pchip(well_ts):
    """
    Perform daily PCHIP interpolation on groundwater well time series data
    
    Args:
        well_ts: DataFrame containing well_id, date, and wte (water table elevation) columns
        
    Returns:
        DataFrame with daily interpolated values for each well
    """
    well_ts = well_ts.copy()
    well_ts['date'] = pd.to_datetime(well_ts['date'])
    well_ts = well_ts.sort_values(['well_id', 'date'])

    interpolated_list = []

    for well_id, group in well_ts.groupby('well_id'):
        # Skip wells with less than 2 observations (minimum required for interpolation)
        if len(group) < 2:
            continue

        # Get date range
        start_date = group['date'].min()
        end_date = group['date'].max()

        # Generate daily date sequence 
        full_dates = pd.date_range(start=start_date, end=end_date, freq='D')

        # Convert dates to ordinal numbers for interpolation
        x_obs = group['date'].map(pd.Timestamp.toordinal)
        y_obs = group['wte'].values

        # Perform PCHIP interpolation
        interpolator = PchipInterpolator(x_obs, y_obs)
        x_new = full_dates.map(pd.Timestamp.toordinal)
        y_new = interpolator(x_new)

        # Create interpolated DataFrame 
        df_interp = pd.DataFrame({
            'well_id': well_id,
            'date': full_dates,
            'wte': y_new
        })

        interpolated_list.append(df_interp)

    interpolated_df = pd.concat(interpolated_list, ignore_index=True)
    return interpolated_df


In [50]:
well_info = pd.read_csv('../data/raw/groundwater/GSLB_1900-2023_wells_with_aquifers.csv')
well_info.columns = well_info.columns.str.lower()


In [51]:
well_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8752 entries, 0 to 8751
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   well_id       8752 non-null   int64  
 1   well_name     8752 non-null   object 
 2   lat_dec       8752 non-null   float64
 3   long_dec      8752 non-null   float64
 4   gse           8752 non-null   float64
 5   aquiferid     8752 non-null   int64  
 6   aquifer_name  8752 non-null   object 
 7   state         8752 non-null   object 
dtypes: float64(3), int64(2), object(3)
memory usage: 547.1+ KB


In [52]:
daily_interp_df = interpolate_daily_pchip(clean_data)
# Merge well information with well_info data
merged_data = pd.merge(daily_interp_df, well_info, on='well_id', how='left')

# Rename columns
merged_data.rename(columns={'lat_dec': 'well_lat', 'long_dec': 'well_lon'}, inplace=True)
merged_data = merged_data[['well_id', 'date', 'wte', 'well_lat', 'well_lon', 'gse']]
# Display merged data
merged_data.head()


,well_id,date,wte,well_lat,well_lon,gse
0,382113113435401,2008-09-03,5395.950000,38.353571,-113.732473,5775.0
1,382113113435401,2008-09-04,5395.950539,38.353571,-113.732473,5775.0
2,382113113435401,2008-09-05,5395.951113,38.353571,-113.732473,5775.0
3,382113113435401,2008-09-06,5395.951721,38.353571,-113.732473,5775.0
4,382113113435401,2008-09-07,5395.952363,38.353571,-113.732473,5775.0


In [53]:
merged_data.to_csv('../data/processed/well_pchip.csv', index=False)